# KNN识别验证码

## 更新sklean

In [2]:
!pip install scikit-learn --upgrade

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 65.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


## 处理数据

验证码资源在 /source 路径下

In [6]:
# -*- coding: utf-8 -*-
"""
Created on 2022/04/14

@author: levin
"""

import numpy as np
import uuid
import os

from PIL import Image


def splitSingle(filename):
    pix = np.array(Image.open(filename).convert('L'))
    # threshold image
    pix = (pix > 135) * 255

    split_parts = [
        [7, 16],
        [20, 29],
        [33, 42],
        [46, 55]
    ]

    for part in split_parts:
        letter = pix[7:, part[0]: part[1]]
        im = Image.fromarray(np.uint8(letter))

        save_path = './letters/' + str(uuid.uuid4()) + '.png'
        print('\t', save_path)
        # im.save(save_path)


def splitAndSave(path):
    path = './source/' + path
    pix = np.array(Image.open(path).convert('L'))
    # threshold image
    pix = (pix > 135) * 255

    split_parts = [
        [7, 16],
        [20, 29],
        [33, 42],
        [46, 55]
    ]

    for part in split_parts:
        letter = pix[7:, part[0]: part[1]]
        im = Image.fromarray(np.uint8(letter))
        save_path = './letters/' + str(uuid.uuid4()) + '.png'
        print('\t', save_path)
        im.save(save_path)


if __name__ == '__main__':

    im_paths = filter(lambda fn: os.path.splitext(fn)[1].lower() == '.png',
    os.listdir('./source'))

    for im_path in im_paths:
        print(im_path)
        splitAndSave(im_path)

6cbd408f-9880-4556-a820-aaab279b14d3.png
	 ./letters/ed7ef1e7-2060-4010-b39c-87dcc4b79f8a.png
	 ./letters/6c89fc98-302a-485f-ae47-044ba6d67e4a.png
	 ./letters/315160d1-4409-4f5b-90ec-e789e6f563f3.png
	 ./letters/fdf5763c-553b-4784-9feb-2fe36f6fbc27.png
66e8cdf4-b181-4906-a903-6df73d7a3003.png
	 ./letters/a3b7f8b1-f54c-4f99-aa54-c84b4f227121.png
	 ./letters/d7301796-1b7f-4196-ad43-eb483d2f42e1.png
	 ./letters/ca12458f-07ad-4b8c-86ad-c258a4863ab5.png
	 ./letters/4a2962bc-e643-4ade-a58a-3493c16f60a4.png
7b0411b0-c7ef-4e7c-8012-ab89955a7a0f.png
	 ./letters/a62263e6-677f-40f4-a40e-ede7380fb47b.png
	 ./letters/93d3a831-cdc1-46cd-9688-42296fb67eaf.png
	 ./letters/df9d983d-453b-47f4-8358-ffd79af014b9.png
	 ./letters/df1fa04f-ad0a-4727-be85-1c6bb4b647cb.png
f6d888ff-4ed8-4819-ae37-85f5617a6cae.png
	 ./letters/183d2af2-d935-4642-8fab-cb63461b5e9e.png
	 ./letters/fc4507b5-a131-42df-b7ad-7889934dd3d4.png
	 ./letters/db34b5e9-4496-4556-b102-9f586b2a5716.png
	 ./letters/015927d3-71c9-4dda-8360-1ebe0

## 训练

In [5]:
# -*- coding: utf-8 -*-
"""
Created on 2022/04/14

@author: levin
"""

from PIL import Image
import numpy as np
import joblib
from sklearn.neighbors import KNeighborsClassifier


def load_dataset():
    X = []
    y = []

    for i in range(70):
        path = "./train/%d%d.png" % (i / 7, i % 7)
        pix = np.asarray(Image.open(path).convert("L"))
        X.append(pix.reshape(9*13))
        y.append(int(i / 7))
    return np.asarray(X), np.asarray(y)


def split_letters(path):
    pix = np.asarray(Image.open(path).convert('L'))
    # threshold image
    pix = (pix > 135) * 255

    split_parts = [
        [7, 16],
        [20, 29],
        [33, 42],
        [46, 55]
    ]
    letters = []
    for part in split_parts:
        letter = pix[7:, part[0]: part[1]]
        letters.append(letter.reshape(9*13))
    return letters


if __name__ == "__main__":
    X, y = load_dataset()
    knn = KNeighborsClassifier()
    knn.fit(X, y)
    joblib.dump(knn, 'knn.job')

## 测试

In [16]:
# -*- coding: utf-8 -*-
"""
Created on 2022/04/14

@author: levin
"""
import numpy as np
import os

from PIL import Image
import joblib


def split_letters(path):
    pix = np.array(Image.open(path).convert('L'))
    # threshold image
    pix = (pix > 135) * 255

    split_parts = [
        [7, 16],
        [20, 29],
        [33, 42],
        [46, 55]
    ]
    letters = []
    for part in split_parts:
        letter = pix[7:, part[0]: part[1]]
        letters.append(letter.reshape(9*13))
    return letters


def get_captcha_result(model_path, filename):
    sipo_knn = joblib.load(model_path)
    letters = split_letters(filename)
    return sipo_knn.predict(letters)


if __name__ == "__main__":
    for test in os.listdir('./test'):
        if test.endswith("png"):
            result = get_captcha_result('knn.job', './test/' + test)
            print("file name: {0}, the classfication result is {1}".format(test,result))

file name: test7.png, the classfication result is [9 3 2 8]
file name: test9.png, the classfication result is [4 9 6 8]
file name: test1.png, the classfication result is [9 7 2 0]
file name: test2.png, the classfication result is [4 5 4 4]
file name: test3.png, the classfication result is [2 0 0 4]
file name: test5.png, the classfication result is [1 4 8 2]
file name: test6.png, the classfication result is [4 7 8 2]
file name: test8.png, the classfication result is [9 8 1 9]
